In [98]:
import pandas as pd
import ast

music_data_frame = pd.DataFrame(index = [1])
music_data_counter = 1

# получение данных о предпочтениях пользователя
user_counter = 1
while user_counter != 40:
	try:
		users_file = open('users_data/' + str(user_counter), 'r', encoding = 'cp1251')
		readed_data = users_file.read()
		users_file.close()
		readed_data = readed_data.lower()

		# преобразуем данные в словарь
		data = ast.literal_eval(readed_data)
		user_music = data['response'][0]['music']
		user_music = user_music.split(',')
		#print(user_music)
		for artist in user_music:
			if (artist[0] == ' '): 
				artist = artist[1:]                
			if (artist not in music_data_frame.columns) and (len(str(artist)) < 20) and (len(str(artist)) > 3):
				music_data_frame[artist] = 0
				music_data_counter = music_data_frame.count(axis = 1)
				music_data_frame.loc[music_data_counter[1]] = 0
		
		for fix_artist in user_music:
			if (fix_artist[0] == ' '):
				fix_artist = fix_artist[1:]  
			for artist in user_music:
				if (artist[0] == ' '):
					artist = artist[1:]                
				
				if (fix_artist != artist) :            
					artist_index = music_data_frame.columns.get_loc(artist)
					fix_artist_index = music_data_frame.columns.get_loc(fix_artist)                   
					music_data_frame[artist][fix_artist_index + 1] += 1
					music_data_frame[fix_artist][artist_index + 1] += 1
               
		user_counter += 1
	except Exception as e:
		#print('errrroooor ' + str(e))
		user_counter += 1
		continue


columns_num = music_data_frame.count(axis = 1)

# сгенерированый csv со всеми взаимосвязями
music_data_frame.to_csv('music_matrix.csv', encoding = 'cp1251')

# обрабатываем матрицу
df = pd.read_csv('music_matrix.csv', encoding = 'cp1251')
df.drop('Unnamed: 0', inplace = True, axis = 1)

#col_list = list(music_data_frame.columns.values)
#print(df[col][1]) 
#print(df.columns[1])    

# чем ближе к 1, тем больше сходство
def tanamoto(vector1, vector2):
	c1, c2, shr = 0, 0, 0
	for i in range(len(vector1)):
		if (vector1[i] != 0) : c1 += 1
		if (vector2[i] != 0) : c2 += 1
		if (vector1[i] != 0) and (vector2[i] != 0) : shr += 1        
	return (float(shr) / (c1 + c2 - shr))

print(list(df[df.columns[3]])) 
print(list(df[df.columns[2]]))
print(tanamoto(list(df[df.columns[3]]), list(df[df.columns[2]])))

[0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0]
[0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0]
0.8
